In [1]:
import pandas as pd
import joblib

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

from expected_disposal_model.data_preparation.preprocessing import convert_chains_to_schema, filter_disposals, create_labels
from expected_disposal_model.config import raw_file_path, preprocessor_file_path, model_v1_file_path, scored_disposal_output

In [2]:
# Load data
chains = pd.read_csv(raw_file_path)
print("Chain data loaded.")

/var/folders/fm/_t15h8ls56q08qggzncfm73w0000gn/T/ipykernel_16093/3602299878.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  chains = pd.read_csv(raw_file_path)


Chain data loaded.


In [3]:
# Processing
preproc = joblib.load(preprocessor_file_path)
chain_features = preproc.transform(chains)

In [4]:
schema_chains = convert_chains_to_schema(chains)
disposals = filter_disposals(schema_chains)

labels = create_labels(chains)

schema_chains = pd.concat([disposals, chain_features, labels], axis=1)
schema_chains = schema_chains.rename(columns={'Disposal':'Disposal_Label'})
print("Preprocessing.. Complete.")

/Users/ciaran/opt/anaconda3/envs/expected-disposal-env/lib/python3.11/site-packages/expected_disposal_model/data_preparation/preprocessing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disposals[ModellingDataContract.RESPONSE] = np.where(disposals['outcome_type'] == 'effective', 1, 0)


Preprocessing.. Complete.


In [5]:
# Load model
exp_disposal_model = joblib.load(model_v1_file_path)

# Scoring Model
schema_chains['xDisposal'] = exp_disposal_model.predict_proba(chain_features)[:, 1]
print("Scoring.. complete.")

/Users/ciaran/opt/anaconda3/envs/expected-disposal-env/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Scoring.. complete.


In [6]:
schema_chains.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,type_Kick_a0,type_Handball_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,Disposal_Label,xDisposal
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,False,True,1,24.0,24.0,9.0,-6.0,-11.0,7.0,-20.0,13.0,23.853721,False,False,True,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,False,False,True,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.001389
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,False,True,1,29.0,29.0,-12.0,5.0,-14.0,2.0,-2.0,-3.0,3.605551,False,False,True,False,False,1,28.0,28.0,-11.0,7.0,-12.0,5.0,-1.0,-2.0,2.236068,False,True,False,True,False,1,24.0,24.0,9.0,-6.0,-11.0,7.0,-20.0,13.0,23.853721,True,False,-1.0,-5.0,0.0,0.0,0.0,1.0,2.0,2.236068,1,0.932960
6,202101_BrisbaneLions_Sydney,1,7.0,1,31.0,31.0,Sydney,George Hewett,-22.0,2.0,6.0,-27.0,Kick,clanger,True,False,1,31.0,31.0,-22.0,2.0,6.0,-27.0,28.0,-29.0,40.311289,False,False,True,False,False,1,30.0,30.0,-14.0,2.0,-22.0,2.0,-8.0,0.0,8.000000,False,True,True,False,False,1,29.0,29.0,-12.0,5.0,-14.0,2.0,-2.0,-3.0,3.605551,True,True,-1.0,-2.0,0.0,0.0,0.0,8.0,0.0,8.000000,0,0.480095
8,202101_BrisbaneLions_Sydney,1,9.0,1,37.0,37.0,Brisbane Lions,Hugh McCluggage,11.0,-26.0,18.0,-23.0,Handball,effective,False,True,1,37.0,37.0,11.0,-26.0,18.0,-23.0,7.0,3.0,7.615773,False,False,True,False,False,1,36.0,36.0,6.0,-27.0,11.0,-26.0,5.0,1.0,5.099020,True,False,False,False,True,1,31.0,31.0,-22.0,2.0,6.0,-27.0,28.0,-29.0,40.311289,True,False,-1.0,-6.0,0.0,0.0,0.0,-5.0,-1.0,5.099020,1,0.948358
16,202101_BrisbaneLions_Sydney,2,17.0,1,93.0,93.0,Brisbane Lions,Oscar McInerney,0.0,0.0,0.0,0.0,Kick,effective,True,False,1,93.0,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,False,False,True,False,False,1,92.0,92.0,-1.0,0.0,0.0,0.0,1.0,0.0,1.000000,False,False,True,False,False,1,40.0,40.0,26.0,-21.0,-0.0,-0.0,-26.0,21.0,33.421550,True,True,-1.0,-53.0,0.0,0.0,0.0,-0.0,-0.0,0.000000,1,0.577267


In [7]:
# Merge back to chains
chains = chains.merge(schema_chains, how = "left", left_on=['Match_ID', 'Chain_Number', 'Order'], right_on=['match_id', 'chain_number', 'order'])


In [8]:
chains.head() 

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,type_Kick_a0,type_Handball_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,Disposal_Label,xDisposal
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,202101_BrisbaneLions_Sydney,1.0,3.0,1.0,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,False,True,1.0,24.0,24.0,9.0,-6.0,-11.0,7.0,-20.0,13.0,23.853721,False,False,True,False,False,1.0,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,False,False,True,False,False,1.0,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.001389
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,202101_BrisbaneLions_Sydney,1.0,5.0,1.0,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,False,True,1.0,29.0,29.0,-12.0,5.0,-14.0,2.0,-2.0,-3.0,3.605551,False,False,True,False,False,1.0,28.0,28.0,-11.0,7.0,-12.0,5.0,-1.0,-2.0,2.236068,False,True,False,True,False,1.0,24.0,24.0,9.0,-6.0,-11.0,7.0,-20.0,13.0,23.853721,True,False,-1.0,-5.0,0.0,0.0,0.0,1.0,2.0,2.236068,1.0,0.932960


In [9]:
# Export data
chains.to_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-disposal-model/data/predictions/disposal_scored_chains.csv", index=False)
print("Exporting.. complete.")

Exporting.. complete.
